In [38]:
import yfinance as yf
import pandas as pd

def download_ticker_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)

    # Flatten MultiIndex columns (if applicable)
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = [f'{col[0]}' for col in data.columns.values]

    # Reset the index to move 'Date' from index to a column
    data = data.reset_index()

    # Convert the 'Date' column to just the date part
    data['Date'] = data['Date'].dt.date

    return data


In [39]:
# Stock tickers and date range
ticker = 'SPY'

start_date = "2021-01-01"
end_date = "2024-01-01"

# Download stock data
ticker_data = download_ticker_data(ticker, start_date, end_date)

# Show the first few rows of the data
ticker_data.head()

[*********************100%***********************]  1 of 1 completed


,Date,Adj Close,Close,High,Low,Open,Volume
0,2021-01-04,349.471680,368.790009,375.450012,364.820007,375.309998,110210800
1,2021-01-05,351.878540,371.329987,372.500000,368.049988,368.100006,66426200
2,2021-01-06,353.982330,373.549988,376.980011,369.119995,369.709991,107997700
3,2021-01-07,359.241547,379.100006,379.899994,375.910004,376.100006,68766800
4,2021-01-08,361.288483,381.260010,381.489990,377.100006,380.589996,71677200


In [40]:
def dollar_cost_average_strategy(data, interval, investment, comm_per_share, comm_min_per_order, comm_max_per_order, platform_fee_per_share, platform_fee_min_per_order, platform_fee_max_per_order):

    invested_amount = 0.0
    cash_balance = 0.0
    num_shares = 0.0
    original_cash = 0.0
    performance_data = []

    for index, data in data[0:].iterrows():
        date = data['Date']
        price = data['Close']
        price_bought_at = 0
        num_shares_bought = 0
        investing_amount = 0

        # Increasing cash balance from pay check
        if index % 20 == 0 and index != 0:
            cash_balance += investment
            original_cash += investment

        # Decrease cash balance to invest
        if index % interval == 0 and cash_balance != 0:
            investing_amount = investment / (20 / interval)
            fees_incurred = 0
            cash_balance -= investing_amount
            invested_amount += investing_amount - fees_incurred
            investing_amount -= fees_incurred
            num_shares += investing_amount / price
            price_bought_at = price
            num_shares_bought = investing_amount / price

        total = cash_balance + num_shares * price

        try:
            t_c = total / original_cash
        except ZeroDivisionError:
            t_c = 1

        performance_data.append((date, original_cash, cash_balance, invested_amount, num_shares * price, total,
                                t_c, investing_amount, price_bought_at, num_shares_bought))

    performance_data = pd.DataFrame(performance_data, columns=['Date', 'Original Cash', 'Cash Balance', 'Capital',
                                                               'Investment', 'Total', 'Total/Cash', 'Investing Amount',
                                                               'Price Bought At', 'Num Shares Bought'])

    performance_data['Performance'] = performance_data['Total/Cash'] - 1

    print("Basic DCA Strategy completed.")

    return performance_data

interval = 20
investment = 100
comm_per_share = 0
comm_min_per_order = 0
comm_max_per_order = 0
platform_fee_per_share = 0
platform_fee_min_per_order = 0
platform_fee_max_per_order = 0
df = dollar_cost_average_strategy(ticker_data, interval, investment, comm_per_share, comm_min_per_order, comm_max_per_order, platform_fee_per_share, platform_fee_min_per_order, platform_fee_max_per_order)
df.head(50)

Basic DCA Strategy completed.


,Date,Original Cash,Cash Balance,Capital,Investment,Total,Total/Cash,Investing Amount,Price Bought At,Num Shares Bought,Performance
0,2021-01-04,0.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000e+00
1,2021-01-05,0.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000e+00
2,2021-01-06,0.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000e+00
3,2021-01-07,0.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000e+00
4,2021-01-08,0.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000e+00
5,2021-01-11,0.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000e+00
6,2021-01-12,0.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000e+00
7,2021-01-13,0.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000e+00
8,2021-01-14,0.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000e+00
9,2021-01-15,0.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000e+00
